In [1]:
import os
import sys
import pandas
import time
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

In [15]:
customer_labels = pandas.read_table('~/Downloads/customer_labels.csv', header=None, names=['cid', 'gender', 'cltv', 'churn'])
cltv_labels = pandas.read_table('~/Documents/labels_cltv.txt', header=None, names=['cid', 'cltv'], sep=' ')

In [16]:
labels_gender = customer_labels[['cid', 'gender']]

In [19]:
labels_churn = customer_labels[['cid', 'churn']]
labels_cltv = cltv_labels[['cid', 'cltv']]

In [20]:
latents = pandas.read_table('~/Downloads/mf_2.csv', header=None, names=['cid', 'latents'])

In [21]:
customers_join_cltv = pandas.merge(latents, cltv_labels, on='cid')

In [22]:
customers_join_churn = pandas.merge(latents, labels_churn, on='cid')
customers_join_gender = pandas.merge(latents, labels_gender, on='cid')

In [23]:
latents = list()
tt = customers_join_churn['latents'].map(lambda x : x.split(' '))
for i in range(0, 194574):
    latents.append([float(yy) for yy in tt[i]])

In [26]:
X = latents
X = np.asarray(X)
Y = customers_join_churn['churn'].tolist()
CID = customers_join_churn['cid'].tolist()

In [30]:
c_list = pandas.read_csv("~/list_c.csv", header=0, names=['idx', 'val'])
c_list = c_list["val"].values
X = np.asarray(X)
Y = np.asarray(Y)
CID = np.asarray(CID)
Y = [1 if yy else 0 for yy in Y]

For finding parameters

In [ ]:

aucs = []
for i in range(0, 10):

    tr_folds_val = pandas.read_csv('tr_folds_val' + str(i) + ".csv", header=0, names=["idx", "val"])
    tr_folds_val = tr_folds_val["val"].values
    tr_folds_tr = pandas.read_csv('tr_folds_tr' + str(i) + ".csv", header=0, names=["idx", "val"])
    tr_folds_tr = tr_folds_tr["val"].values

    c_ids_val = c_list[tr_folds_val]
    c_ids_tr = c_list[tr_folds_tr]
    xsorted = np.argsort(CID)
    ypos = np.searchsorted(CID[xsorted], c_ids_val)
    idx_val = xsorted[ypos]
    ypos = np.searchsorted(CID[xsorted], c_ids_tr)
    idx_tr = xsorted[ypos]

    LR = LogisticRegression()
    LR.fit(X[idx_tr], Y[idx_tr, j])
    auc_val = roc_auc_score(Y[idx_val, j], LR.predict_proba(X[idx_val])[:, 1])
    pandas.DataFrame(LR.predict_proba(X[idx_val])[:, 1]).to_csv('val_scores' + str(i) + ".csv")
    pandas.DataFrame(Y[idx_val]).to_csv('val_scores_true' + str(i) + ".csv")
    print "AUC on VAL: " + str(auc_val)
    aucs.append(auc_val)
print(np.mean(aucs))

CLTV

In [674]:
X = latents
X = np.asarray(X)
Y = customers_join_cltv['cltv'].tolist()
CID = customers_join_churn['cid'].tolist()

[3, 5, 2, 0, 6, 0, 3, 2, 6, 0, 2, 4, 2, 5, 5]

In [675]:
Y = label_binarize(Y, classes=[0, 1, 2, 3, 4, 5, 6])

In [676]:
Y.shape

(194574, 7)

In [712]:
c_list = pandas.read_csv("~/list_c.csv", header=0, names=['idx', 'val'])
c_list = c_list["val"].values
X = np.asarray(X)
Y = np.asarray(Y)
CID = np.asarray(CID)

In [713]:
for j in range(0, 7):
    aucs = []
    for i in range(0, 10):

        tr_folds_te = pandas.read_csv('tr_folds_te' + str(i) + ".csv", header=0, names=["idx", "val"])
        tr_folds_te = tr_folds_te["val"].values
        tr_folds_tr = pandas.read_csv('tr_folds_tr' + str(i) + ".csv", header=0, names=["idx", "val"])
        tr_folds_tr = tr_folds_tr["val"].values

        c_ids_te = c_list[tr_folds_te]
        c_ids_tr = c_list[tr_folds_tr]
        xsorted = np.argsort(CID)
        ypos = np.searchsorted(CID[xsorted], c_ids_te)
        idx_te = xsorted[ypos]
        ypos = np.searchsorted(CID[xsorted], c_ids_tr)
        idx_tr = xsorted[ypos]

        LR = LogisticRegression()
        LR.fit(X[idx_tr], Y[idx_tr, j])
        auc_te = roc_auc_score(Y[idx_te, j], LR.predict_proba(X[idx_te])[:, 1])
        pandas.DataFrame(LR.predict_proba(X[idx_te])[:, 1]).to_csv('te_scores' + str(i) + ".csv")
        pandas.DataFrame(Y[idx_te]).to_csv('te_scores_true' + str(i) + ".csv")
        print "AUC on TE: " + str(auc_te)
        aucs.append(auc_te)
    print(np.mean(aucs))

0.720812490687
